# Agentic Essay Writer

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = MemorySaver()

In [3]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [4]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-5-nano", temperature=0)

In [5]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""

RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [7]:
from pydantic import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [8]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [9]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

In [10]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [12]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [13]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [14]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [15]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [16]:
builder = StateGraph(AgentState)

builder.add_node("planner", plan_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_critique", research_critique_node)

builder.set_entry_point("planner")

builder.add_conditional_edges("planner", should_continue, {END: END, "reflect": "reflect"})

builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

graph=builder.compile(checkpointer=memory)

In [17]:
thread = {"configurable": {"thread_id": "1"}}
initial_state = {
    'task': "What is the best way to plan to build a house from scratch",
    'plan': "",
    'draft': "",
    'critique': "",
    'content': [],
    'max_revisions': 2,
    'revision_number': 1,
}

for s in graph.stream(initial_state, thread):
    print(s)

{'planner': {'plan': 'Here is a high-level essay outline you can use to organize a piece on “the best way to plan to build a house from scratch.” It includes section prompts and notes to guide your writing.\n\nWorking thesis (to shape the essay)\n- A successful build from scratch rests on a disciplined, phased planning framework that starts with clear goals and a realistic budget, followed by thorough site analysis, informed design, robust permitting and financing plans, careful team selection, and strong project management—integrating risk, sustainability, and quality control to deliver on time and within budget.\n\nOutline with notes for each section\n\n1) Introduction\n- What to cover:\n  - Hook that illustrates the stakes of planning a house (costs, timelines, quality of life).\n  - Why planning matters more than “dreaming big” without structure.\n  - Present the core framework the essay will advocate (goal-setting → site/design → permits/financing → team/procurement → construction

In [18]:
state = []
for s in graph.get_state_history(thread):
    state.append(s)
    print(s)

StateSnapshot(values={'task': 'What is the best way to plan to build a house from scratch', 'plan': 'Here is a high-level essay outline you can use to organize a piece on “the best way to plan to build a house from scratch.” It includes section prompts and notes to guide your writing.\n\nWorking thesis (to shape the essay)\n- A successful build from scratch rests on a disciplined, phased planning framework that starts with clear goals and a realistic budget, followed by thorough site analysis, informed design, robust permitting and financing plans, careful team selection, and strong project management—integrating risk, sustainability, and quality control to deliver on time and within budget.\n\nOutline with notes for each section\n\n1) Introduction\n- What to cover:\n  - Hook that illustrates the stakes of planning a house (costs, timelines, quality of life).\n  - Why planning matters more than “dreaming big” without structure.\n  - Present the core framework the essay will advocate (g

In [23]:
print(state[0].values["draft"])

Planning to build a house from scratch is as much about disciplined process as it is about big ideas. The most successful builds start with a clear goal, a realistic budget, and a phased plan that unfolds step by step—from choosing a site and defining the program to securing financing, assembling the right team, and guiding construction to completion. The best approach, then, is a structured framework that integrates design, cost awareness, permitting, procurement, and rigorous project management, all while keeping an eye on risk, sustainability, and long-term quality. This essay argues that a disciplined, phased planning method—centered on goals, site and design, permits and financing, and strong execution—delivers the project on time and within budget.

First, establish goals, constraints, and the program. Begin by separating must-haves from nice-to-haves and translating family size, lifestyle, and future needs into a concrete program. Map site characteristics such as lot size, orien